In [27]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [28]:
import numpy as np
import pandas as pd
import csv

# 데이터
##### df_product
##### df_openmarket
##### df_link

In [29]:
product_data = pd.read_csv('./data/product.csv').fillna('없음')
df_product = pd.DataFrame(product_data)

In [30]:
openmarket_data = pd.read_csv('./data/test_openmarket.csv', error_bad_lines=False).fillna('없음')
df_openmarket = pd.DataFrame(openmarket_data)
# df_openmarket = df_openmarket.rename(columns = {'제휴사 코드':'AFcode', '상품 코드':'OMcode'})"

In [31]:
# df_openmarket = df_openmarket.head(1000)
# df_openmarket

In [32]:
#링크데이터
link_data = pd.read_csv('./data/linkinfo.csv')
df_link = pd.DataFrame(link_data)

# 테스트를 위한 데이터 추출

In [33]:
import time
start = time.time()

product_list = [] #입력 100개에 대한 정답 내부데이터\n",
for openmarket_code in df_openmarket['상품 코드']:
    temp = []
    link = df_link[df_link['제휴사 상품 코드'] == openmarket_code]
    product_code = link.iloc[0,0]
    code = df_product[df_product['상품 코드'] == product_code]
    temp.append(code['상품 코드'].iloc[0])
    temp.append(code['상품명'].iloc[0])
    temp.append(code['제조사'].iloc[0])
    temp.append(code['브랜드'].iloc[0])
    temp.append(code['이미지 URL'].iloc[0])
    temp.append(code['대분류 카테고리명'].iloc[0])
    temp.append(code['중분류 카테고리명'].iloc[0])
    temp.append(code['소분류 카테고리명'].iloc[0])
    temp.append(code['세분류 카테고리명'].iloc[0])
    temp.append(code['간략설명'].iloc[0])
    product_list.append(temp)
#     print(df_product['상품 코드'])
print ('Cell took %.2f seconds to run.' %(time.time() - start))

Cell took 27.55 seconds to run.


In [34]:
# product_list

In [35]:
df_product = pd.DataFrame(product_list, columns=["상품 코드","상품명","제조사","브랜드","이미지 URL","대분류 카테고리명","중분류 카테고리명","소분류 카테고리명","세분류 카테고리명","간략설명"])
# df_product

# word2vec

### 상품명, 단어 추출
##### 상품명: product_name, openmarket_name
##### 단어: product_words, openmarket_words

In [36]:
product_array = df_product.to_numpy()
openmarket_array = df_openmarket.to_numpy()

#### 상품명 제품코드 제거

In [37]:
import re
p = re.compile('[a-zA-Z]*[0-9]+[a-zA-Z0-9]*[-]?[_]?[s]?[a-zA-Z]*[0-9]+[a-zA-Z0-9]*[-]?[_]?[s]?[a-zA-Z]*[0-9]+[a-zA-Z0-9]*')
for i, product in enumerate (product_array):
    target = product[1].split(" ")
    new_str = []
    for kk in target:
        matchname = p.match(kk)
        if matchname == None:
            new_str.append(kk)
            new_str.append(" ")
    new_str = ''.join(new_str)
#     print(product_array[i][1])
#     print(new_str)
    product_array[i][1] = new_str


openmarket_list = []
for i, openmarket in enumerate(openmarket_array):
    target = openmarket[2].split(" ")
#     print(target)
    new_str = []
    for kk in target:
        matchname = p.match(kk)
        if matchname == None:
            new_str.append(kk)
            new_str.append(" ")
    new_str = ''.join(new_str)
#     print(openmarket_array[i][2])
#     print(new_str)
    openmarket_array[i][2] = new_str

In [38]:
def blankSearch(name, nameLen, stack, blanklist, templist):
    for i in range(nameLen):
        if name[i] == '(':
            stack.append(i)
            templist.append(i)
        elif name[i] == ')' and len(stack) != 0:
            blanklist.append(stack.pop()),
            blanklist.append(i)

    stack = []
    templist = []
    for i in range(nameLen):
        if name[i] == '[':
            stack.append(i)
            templist.append(i),
        elif name[i] == ']' and len(stack) != 0:
            blanklist.append(stack.pop())
            blanklist.append(i)

In [39]:
codelist = [] # 제품코드 추출할 리스트
printlist = []
for i, openmarket in enumerate (openmarket_array):
    name = openmarket[2] # 상품명
    nameLen = len(name) # 상품명 길이

    # 괄호 구하기 위한 리스트
    stack = []
    blanklist = []
    templist = []
    
    blankSearch(name, nameLen, stack, blanklist, templist)
    # 정규식 사용하여 제품코드 추출하기
    blankLen = len(blanklist)
    blankLen = (int)(blankLen/2)
    linelist = []
    for k in range(blankLen):   # 괄호만큼 돈다.
        pre = blanklist[k*2]
        post = blanklist[k*2+1]
        tempname = name[pre:post+1]
        printlist.append(tempname)
    for word in printlist:
        name = name.replace(word, " ")
    name = name.lstrip()
    openmarket_array[i][2] = name
    openmarket_array[i][2]


In [40]:
import re
df_name = df_openmarket.iloc[:,[2]]
# print(openmarket_array)
# df_sample2 = df.iloc[:,[1]]
# df_len = len(df_name)
df_len = len(openmarket_array)
cnt = 0
example=df_name.head(df_len)
p = re.compile('[a-zA-Z]*[0-9]+[a-zA-Z0-9]*[_]?[W]?[a-zA-Z]*[0-9]+[a-zA-Z0-9]*[_]?[W]?[a-zA-Z]*[0-9]+[a-zA-Z0-9]*')
# p = re.compile('[^ ㄱ-ㅣ가-힣]')
answer= []
while cnt != df_len:
    name = openmarket_array[cnt][2]
#     print(name)
    cnt = cnt+1
    res = p.search(name)
#     print(name)
#     print(res)
    if res != None and len(res.group()) >= 5:
        name = name.replace(res.group(), " ")
    openmarket_array[cnt-1][2] = name


In [41]:
relists = []
f = open("./relist.txt", 'r', encoding='utf-8')
lines = f.read().splitlines()
for line in lines:
    relists.append(line)
f.close()

In [42]:
for i in range(len(openmarket_array)):
    pro_name = openmarket_array[i][2]
    for relist in relists:
        reg = re.compile(relist)
        searching = reg.search(pro_name)
        if searching != None:
            if searching.group() != '자카드' or searching.group() != '쟈카드':
                openmarket_array[i][2] = pro_name.replace(searching.group(), ' ')

In [43]:
notuses = []
f = open("./notuse.txt", 'r',encoding='utf-8')
lines = f.read().splitlines()
for line in lines:
    notuses.append(line)
f.close()


In [44]:
brandlists = np.array(df_product['브랜드'])
brandlists
len(brandlists)

100

In [45]:
brandlist_set = set(brandlists)
brandlist_set
len(brandlist_set)

18

In [46]:
for i in range(len(openmarket_array)):
    pro_name = openmarket_array[i][2]
    for notuse in notuses:
        if notuse.upper() in pro_name.upper():
            pro_name = pro_name.upper().replace(notuse.upper(), ' ')
    for brandlist in brandlists:
        if brandlist.upper() in pro_name.upper():
            pro_name = pro_name.upper().replace(brandlist.upper(), ' ')
    pro_name = ' '.join(pro_name.split())
    openmarket_array[i][2] = pro_name

In [47]:
for t in product_array:
    print(t[1])

남성 쿨 배쏙티 2단계 보정나시 (블랙) 
슬림 보정 올인원 
복부 이중 와이어 A컵 치마 올인원 
런닝형 소프트타입 바디슈트 
웨이브헴 문직샤틴 숏거들 
파워망 초 슬림 누디 사각거들 
복부 이중 와이어 A컵 치마 올인원 
허리 4cm밴드 보정 니퍼 
마미써포트 산전용 거들 
라이크라 올인원 (라이트브라운) 
와이어 올인원 바디슈트 
문직사틴 햄 소프트 삼각거들 (블랙) 
문직사틴 햄 소프트 삼각거들 (블랙) 
인견 무빙 올인원 
노몰드 와이어 올인원 
기능성 올인원 
면 헴소재 소프트 팬티형 거들 
파워넷 무봉제 삼각거들 
런닝형 풀컵 바디쉐이퍼 
트리플 밸런스업 지퍼란쥬팬티 2종세트 
하드타입 숏거들 
사이드 힙패드 사각거들 
자가드 라이크라 쉐이퍼 
런닝형 소프트타입 바디슈트 
ONCORE 하이웨스트 거들 
소프트 타입 스탠다드 사각거들 
트리플보정 지퍼란쥬4종 팬티8종 풀패키지 RO29/RHP02 
인견 란쥬 쉐이퍼 4세트 
하드타입 풀컵 올인원 
슬림 보정 올인원 
파워넷 레이스 복대 거들 
지퍼 치마쉐이퍼 
레이스 복대 
허니 브라이덜 가터 
런닝형 지퍼 올인원 
런닝형 지퍼 올인원 
보정 바디쉐이퍼 
기능성 힙업 삼각 뽕거들 
문직 파워네트 바디쉐이퍼 
하드타입 풀컵 올인원 
소프트 쇼트거들 
여성 스키나 거들삼각 (블랙) 
런닝형 지퍼 올인원 
사이드 힙패드 사각거들 
문직 파워네트 바디쉐이퍼 
파워샤틴 사각거들 
모달 컵란쥬 
베리섹시 가터 벨트 
베리섹시 가터 벨트 
미듐타입 문직 사틴 올인원 
파워샤틴 사각거들 
파워네트원단 웨이스트니퍼 
복부 이중 와이어 A컵 치마 올인원 
기능성 힙업 사각 뽕거들 
올인원 바디수트 
파워샤틴 사각거들 
하이웨이스트 레이스 자가드 사각거들 
파워네트 올인원 
허니 브라이덜 가터벨트 
하이플렉스 바디쉐이퍼 
하드타입 스탠다드 사각거들 
미디움 쇼트 거들 (스킨) 
하드타입 풀컵 올인원 
기능성 힙업 사각 뽕거들 
풀컵 하드타입 런닝형 올인원 
슬림 보정 올인원 
모달 컵란쥬 
내추럴 케어 풀컵 올인원 
하

In [48]:
for t in openmarket_array:
    print(t[2])

WOX 남성 쿨 배쏙티 2단계 보정속옷 보정나시 블랙
강한원단으로 몸매를 예쁘게 런닝형 쉐이퍼
GG3zS2 이중 와이어 B컵 치마올인원
샬루트 런닝형 소프트 타입 바디 쉐이퍼
문직 샤틴 볼륨업 삼각거들
WS IL 파워망 초 슬림 누디 사각거들
SW 매쉬 B컵 빅사이즈 치마 올인원
허리4CM밴드 보정니퍼 64
자스민 마미써포트 산전용 거들 L.JGR
라이크라 뷰티 기능성 올인원
파워넷 상체보정힙업 올인원 바디슈트
미듐타입 쇼트거들
미듐타입 쇼트거들
80B 95B/인견 무빙 올인원 바디슈트1종
노몰드와이어 A컵 올인원
LV 기능성 올인원 1종
면 햄소재 팬티형 삼각거들
통기성이 좋은 파워넷 무봉제 삼각거들
우아한 플라워 미디움타입 문직 사틴 패드 쉐이퍼 1택C컵D컵
트리플 밸런스업 지퍼란쥬팬티 세트
햄 파워네트 하드타입 삼각거들
DB 사이드 힙패드거들 골반거들
체형보정 라이크라 바디슈트 치마올인원
분당점 A B컵 여성 런닝형 살루트 소프트 올인원
O ORE 하이웨스트 미드따이 쇼츠 강한보정 거들
힙업 소프트 거들
트리플보정 지퍼란쥬4종 팬티8종 풀패키지
라미카 인견 보정 란쥬쉐이퍼 4세트 패키지 보정속옷
CD컵 보정력이 좋은 올인원
DB 강한 몸매보정 런닝형 파워 올인원
파워넷 레이스 복대 거들
지퍼 치마쉐이퍼
날씬한 허리라인 레이스 복대
허니 브라이덜 가터벨트
하드타입 앞지퍼 탄탄보정 올인원 CD
런닝형 지퍼 바디수트 AB컵
DB 보정풀컵런닝형바디쉐이퍼
DB 자연스러운 엉덩이볼륨 뽕패드 힙거들
A B컵 여성 문직 파워네트 바디쉐이퍼
E컵 자카드 햄 파워네트 하드타입 올인원
소프트쇼트거들
여성 스키나 거들삼각BK
런닝형 지퍼올인원
여성 사이드 힙패드 거들
여성 문직 파워네트 바디쉐이퍼
아랫배를 엉덩이 사각거들
패드브라런닝
베리 섹시 가터벨트
베리 섹시 가터벨트
풀컵 미듐타입 문직 사틴 올인원C D컵
사각거들
기능성 지퍼형니퍼
eR3tb5 B컵치마올인원 이중복부 빅사이즈
완벽한 힙볼륨업 보정형 힙패드 사각거들
가볍고 시원한 다이어트 보정 올인원
사각거들거들사각거들보

### 외부데이터 불용어 제거

### 상품명 word 자르기

In [49]:
product_words = []
openmarket_words = []

for text in product_array:
    for word in text[1].split(' '):
        product_words.append(word)
product_words = set(product_words)
print("product_words = ", len(product_words))

for text in openmarket_array:
    for word in text[2].split(' '):
        openmarket_words.append(word)
# openmarket_words
openmarket_words = set(openmarket_words)
# openmarket_words
print("openmarket_words = ", len(openmarket_words))

product_words =  129
openmarket_words =  237


### Word2Vec을 위한 상품명 단어 split

In [50]:
product_search = []
product_sentences = []
for sentence in product_array:
#     print(sentence[1])
    product_code = []
    product_code.append(sentence[0])
    product_code.append(sentence[1].lower().split())
    product_sentences.append(sentence[1].lower().split())
    product_search.append(product_code)
# print(product_sentences)

openmarket_search = []
openmarket_sentences = []
for sentence in openmarket_array:
    openmarket_code = []
    openmarket_code.append(sentence[1])
    openmarket_code.append(sentence[2].lower().split())
    openmarket_sentences.append(sentence[2].lower().split())
    openmarket_search.append(openmarket_code)
# print(openmarket_search)

totalsearch = []
totalsearch = product_search + openmarket_search
totalsentences = product_sentences + openmarket_sentences
# totalsearch

In [51]:
product_code = []
product_sentences = []
for sentence in product_array:
#     print(sentence[1])
    product_code.append(sentence[0])
    product_sentences.append(sentence[1].lower().split())
#     print(product_code)
#     print(product_sentences)


openmarket_code = []
openmarket_sentences = []
for sentence in openmarket_array:
    openmarket_code.append(sentence[1])
    openmarket_sentences.append(sentence[2].lower().split())
#     print(openmarket_code)
#     print(openmarket_search)

# totalsearch = []
# totalsearch = product_search + openmarket_search
# totalsentences = product_sentences + openmarket_sentences
# totalsearch

### Word2Vec

In [52]:
import gensim
print(gensim.__version__)

C:\ProgramData\Anaconda3\envs\newPJT3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


3.4.0


In [53]:
from gensim.models import Word2Vec
model = Word2Vec(totalsentences, window=8, min_count=1, workers=4)
model

## WMD

In [54]:
import pyemd
print(pyemd.__version__)

0.5.1


In [55]:
import time
import os
start = time.time()

from gensim.models import Word2Vec

result = []

for j, openmarket in enumerate(openmarket_search):
    min = 1
    
    res_openmarketindex = openmarket[0]
    res_openmarketname = openmarket[1]
    res_productname = ""
    res_productidnex = -1
    res_distance = 1
    
    for i, product in enumerate(product_search):
        distance = model.wmdistance(openmarket[1], product[1])
        if(min > distance):
            min = distance
            res_productname = product[1]
            res_productindex = product[0]
            res_distance = distance
    
    res_openmarketname = ' '.join(res_openmarketname)
    res_productname = ' '.join(res_productname)
    
    temp = []
    result.append([res_openmarketindex,res_openmarketname,res_productindex,res_productname,res_distance])
    
print ('Cell took %.2f seconds to run.' %(time.time() - start))

C:\ProgramData\Anaconda3\envs\newPJT3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


Cell took 4.20 seconds to run.


### 정답 상품 찾기

In [56]:
res_pcode = []
res_pname = []
res_check = []
for target in result:
    openmarket_code = target[0]
    link = df_link[df_link['제휴사 상품 코드'] == openmarket_code]
    product_code = link.iloc[0,0]
    pname = df_product[df_product['상품 코드'] == product_code]['상품명']
    res_pcode.append(product_code)
    res_pname.append(pname.iloc[0])
    if(target[2] == product_code):
        res_check.append(True)
    else:
        res_check.append(False)

In [57]:
# result

In [58]:
df_result = pd.DataFrame(result, columns=["외부 상품 코드","외부 상품명","비교 상품 코드","비교 상품명","거리"])
df_result.insert(4,'실 내부 코드',res_pcode)
df_result.insert(5,'실 내부 상품명',res_pname)
df_result.insert(6,'일치 여부',res_check)

In [59]:
print(len(df_result[df_result['일치 여부'] == True])/len(result)*100, "%")

46.0 %


In [60]:
all = 0
count = 0
distance = 1
for find in range(len(df_result)):
    if(df_result['거리'].iloc[find] < distance):
        if df_result['일치 여부'].iloc[find]:
            count +=1
        all += 1
print(len(df_result),"개 중 분류된 상품", all)
# print("all = ", all)
# print("count = ", count)
print("신뢰도",count/all, "%")

100 개 중 분류된 상품 100
신뢰도 0.46 %


In [62]:
df_result

,외부 상품 코드,외부 상품명,비교 상품 코드,비교 상품명,실 내부 코드,실 내부 상품명,일치 여부,거리
0,1760821404,wox 남성 쿨 배쏙티 2단계 보정속옷 보정나시 블랙,12412565,남성 쿨 배쏙티 2단계 보정나시 (블랙),12412565,남성 쿨 배쏙티 2단계 보정나시 (블랙),True,0.015213
1,B879729036,강한원단으로 몸매를 예쁘게 런닝형 쉐이퍼,11146647,런닝형 풀컵 바디쉐이퍼,11915329,슬림 보정 올인원 VBTH102,False,0.031912
2,1336929889_1336929889,gg3zs2 이중 와이어 b컵 치마올인원,4054078,복부 이중 와이어 a컵 치마 올인원,4054078,복부 이중 와이어 A컵 치마 올인원 BS6003,True,0.026601
3,6741830350,샬루트 런닝형 소프트 타입 바디 쉐이퍼,10712463,소프트 타입 스탠다드 사각거들,10991340,런닝형 소프트타입 바디슈트 DBT0605,False,0.026648
4,1826792024,문직 샤틴 볼륨업 삼각거들,11209521,미듐타입 문직 사틴 올인원,11592290,웨이브헴 문직샤틴 숏거들 VGRM100S,False,0.029527
...,...,...,...,...,...,...,...,...
95,1260433772_1260433772,kn5kr5 올인원 치마 파워넷 노몰드,4054329,파워넷 노몰드 치마 올인원,4054329,파워넷 노몰드 치마 올인원 BS6807,True,0.008632
96,7256698418,히트 배쏙티 1단계 보정속옷 보정나시 버건디,12461549,히트 배쏙티 1단계 (버건디),12461549,히트 배쏙티 1단계 (버건디),True,0.020742
97,2839639641,"c,d컵 풀컵 하드타입 런닝형 어깨끈 올인원",11665811,풀컵 하드타입 런닝형 올인원,11665811,풀컵 하드타입 런닝형 올인원 VBTH100,True,0.013642
98,1711039542,크레오라 모달 쉐이핑 집업 란쥬 팬티 8종,10342341,크레오라 모달 쉐이핑 집업 란쥬 팬티 8종 세트,10342341,크레오라 모달 쉐이핑 집업 란쥬 팬티 8종 세트 MR037,True,0.005142
